# Illustrating an issue when slicing xarray.DataArray with chunks

 - **author** : Julien Le Sommer, CNRS
 - **date** : June 2016
 - **context** : tests following an issue when sliding grid objects, see https://github.com/lesommer/oocgcm/issues/33
 - **key conclusion** : lock=False + dask chunk : does not allow arithmetic operations with xarray.DataArray 

## Modules imports and versions

In [15]:
# modules
import dask
import xarray as xr
print "dask version : ",dask.__version__
print "xarray version : ",xr.__version__

dask version :  0.9.0
xarray version :  0.7.2


## Desciption of the NetCDF file

In [2]:
# file
coordfile  = '/Users/lesommer/data/NATL60/NATL60-I/NATL60_coordinates_v4.nc'

In [19]:
!ncdump -hs /Users/lesommer/data/NATL60/NATL60-I/NATL60_coordinates_v4.nc

netcdf NATL60_coordinates_v4 {
dimensions:
	y = 3454 ;
	x = 5422 ;
variables:
	double e1f(y, x) ;
		e1f:missing_value = 1.e+20 ;
	double e1t(y, x) ;
		e1t:missing_value = 1.e+20 ;
	double e1u(y, x) ;
		e1u:missing_value = 1.e+20 ;
	double e1v(y, x) ;
		e1v:missing_value = 1.e+20 ;
	double e2f(y, x) ;
		e2f:missing_value = 1.e+20 ;
	double e2t(y, x) ;
		e2t:missing_value = 1.e+20 ;
	double e2u(y, x) ;
		e2u:missing_value = 1.e+20 ;
	double e2v(y, x) ;
		e2v:missing_value = 1.e+20 ;
	double glamf(y, x) ;
		glamf:missing_value = 1.e+20 ;
	double glamt(y, x) ;
		glamt:missing_value = 1.e+20 ;
	double glamu(y, x) ;
		glamu:missing_value = 1.e+20 ;
	double glamv(y, x) ;
		glamv:missing_value = 1.e+20 ;
	double gphif(y, x) ;
		gphif:missing_value = 1.e+20 ;
	double gphit(y, x) ;
		gphit:missing_value = 1.e+20 ;
	double gphiu(y, x) ;
		gphiu:missing_value = 1.e+20 ;
	double gphiv(y, x) ;
		gphiv:missing_value = 1.e+20 ;
	float nav_lat(y, x) ;
		nav_lat:units = "degrees_north" ;
		nav_lat:valid

## Defining chunks and slice

In [4]:
# chunks
chunks = (1727,2711)
xr_chunks = {'x': chunks[-1], 'y': chunks[-2]}
xr_chunks_t = {'x': chunks[-1], 'y': chunks[-2],'time_counter':1}

In [16]:
# slice
from oocgcm.core.utils import returnslice
zone = returnslice[...,500:1500,1500:2500] # getting the slice corresponding to [...,500:1500,1500:2500]

## Illustrating the issue

### Case 1. : xarray.DataArray with chunk=None, lock=False
This works fine.

In [6]:
_e1t = xr.open_dataset(coordfile,chunks=None,lock=False)['e1t']
_e2t = xr.open_dataset(coordfile,chunks=None,lock=False)['e2t']

In [7]:
e1t = _e1t[zone] # a 2d array defining grid cell size in x direction
e2t = _e2t[zone] # a 2d array defining grid cell size in y direction
dxdy = e1t * e2t # a 2d array defining grid cell area 
print dxdy[0,0].to_masked_array()

2344294.78069


### Case 2. : xarray.DataArray with chunk=None, lock=True
This works fine.

In [8]:
_e1t = xr.open_dataset(coordfile,chunks=None,lock=True)['e1t']
_e2t = xr.open_dataset(coordfile,chunks=None,lock=True)['e2t']

In [9]:
e1t = _e1t[zone] # a 2d array defining grid cell size in x direction
e2t = _e2t[zone] # a 2d array defining grid cell size in y direction
dxdy = e1t * e2t # a 2d array defining grid cell area 
print dxdy[0,0].to_masked_array()

2344294.78069


### Case 3. : xarray.DataArray with chunks, lock=True
This works fine.

In [10]:
_e1t = xr.open_dataset(coordfile,chunks=xr_chunks,lock=True)['e1t']
_e2t = xr.open_dataset(coordfile,chunks=xr_chunks,lock=True)['e2t']

In [11]:
e1t = _e1t[zone] # a 2d array defining grid cell size in x direction
e2t = _e2t[zone] # a 2d array defining grid cell size in y direction
dxdy = e1t * e2t # a 2d array defining grid cell area 
print dxdy[0,0].to_masked_array()

2344294.78069


### Case 4. : xarray.DataArray with chunks, lock=False
This raises an error.

In [12]:
_e1t = xr.open_dataset(coordfile,chunks=xr_chunks,lock=False)['e1t']
_e2t = xr.open_dataset(coordfile,chunks=xr_chunks,lock=False)['e2t']

In [17]:
e1t = _e1t[zone] # a 2d array defining grid cell size in x direction
e2t = _e2t[zone] # a 2d array defining grid cell size in y direction
dxdy = e1t * e2t # a 2d array defining grid cell area 
print dxdy[0,0].to_masked_array()

TypeError: unsupported operand type(s) for *: 'LazilyIndexedArray' and 'LazilyIndexedArray'

Traceback
---------
  File "/Users/lesommer/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/Users/lesommer/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/Users/lesommer/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
